In [1]:
import tensorflow as tf
import pandas as pd 
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
from glob import glob

In [2]:
# Define Constants by re-sizing all the images
IMAGE_SIZE = [224, 224]

train_path = "C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset"

In [3]:
# Import the InceptionV3 model and here we will be using imagenet weights

mobilenet = MobileNetV2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:
# We don't need to train existing weights
for layer in mobilenet.layers:
    layer.trainable = False

In [5]:
# Folders in the Training Set
folders = glob('C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train\\*')
folders

['C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train\\freshapples',
 'C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train\\freshbanana',
 'C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train\\freshoranges',
 'C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train\\rottenapples',
 'C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train\\rottenbanana',
 'C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train\\rottenoranges']

In [6]:
# Model layers -> can add more if required
x = Flatten()(mobilenet.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [7]:
# Create a model object
model = Model(inputs=mobilenet.input, outputs=prediction)

# View the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [8]:
# Defining the cost and model optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [9]:
# Using the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
# Training Generator
training_set = train_datagen.flow_from_directory('C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 10901 images belonging to 6 classes.


In [11]:
# Testing Generator
test_set = test_datagen.flow_from_directory('C:\\Users\\rashm\\Downloads\\archive (7)\\dataset\\dataset\\test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 2698 images belonging to 6 classes.


In [ ]:
# fit the model, it will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\rashm\AppData\Local\Temp\ipykernel_16680\2655456400.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
341/341 [==============================] - 226s 605ms/step - loss: 0.5051 - accuracy: 0.9404 - val_loss: 0.3781 - val_accuracy: 0.9603
Epoch 2/50
341/341 [==============================] - 151s 443ms/step - loss: 0.2703 - accuracy: 0.9755 - val_loss: 0.1432 - val_accuracy: 0.9889
Epoch 3/50
341/341 [==============================] - 125s 367ms/step - loss: 0.1785 - accuracy: 0.9853 - val_loss: 0.1804 - val_accuracy: 0.9893
Epoch 4/50
341/341 [==============================] - 124s 364ms/step - loss: 0.1722 - accuracy: 0.9859 - val_loss: 0.3362 - val_accuracy: 0.9867
Epoch 5/50
341/341 [==============================] - 125s 366ms/step - loss: 0.1380 - accuracy: 0.9900 - val_loss: 0.2601 - val_accuracy: 0.9878
Epoch 6/50
341/341 [==============================] - 123s 361ms/step - loss: 0.1017 - accuracy: 0.9922 - val_loss: 0.2143 - val_accuracy: 0.9885
Epoch 7/50
341/341 [==============================] - 131s 383ms/step - loss: 0.1068 - accuracy: 0.9922 - val_loss: 0.4961 -

In [ ]:
# Plot the Loss and Accuracy
# Loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# Accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from tensorflow.keras.models import load_model
model.save('C:\\Users\\rashm\\anaconda3\\CropDisease\\Fruit_model_mobilenetv2.h5')